In [1]:
import os,tensorflow as tf
os.chdir("../")

In [2]:
import mlflow

In [13]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/kumarashutoshbtech2023/dl_mlops.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "kumarashutoshbtech2023"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "c15604c69f0ae3b57c8400896b754673666d28a4"

mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
mlflow.set_experiment("test-experiment")  # will auto-create

with mlflow.start_run():
    mlflow.log_param("lr", 0.01)
    mlflow.log_metric("accuracy", 0.95)

print("✅ Logged successfully!")

2025/10/18 01:47:12 INFO mlflow.tracking.fluent: Experiment with name 'test-experiment' does not exist. Creating a new experiment.


🏃 View run flawless-hare-359 at: https://dagshub.com/kumarashutoshbtech2023/dl_mlops.mlflow/#/experiments/1/runs/9222c9c479564c6bb8104dd2989f0996
🧪 View experiment at: https://dagshub.com/kumarashutoshbtech2023/dl_mlops.mlflow/#/experiments/1
✅ Logged successfully!


In [14]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from src.cnn.constants import *
from src.cnn.utils.common import read_yaml, create_directories, save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/entbappy/chest-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [19]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("CNN-Chest-CT-Scan")

        with mlflow.start_run():
            # Log all hyperparameters
            mlflow.log_params(self.config.all_params)

            # Log metrics (ensure you set these earlier in your eval code)
            mlflow.log_metrics({
                "accuracy": self.model_accuracy,
                "loss": self.model_loss
            })

            # Log model
            mlflow.keras.log_model(self.model, "model")

In [26]:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

    # Directly start a run without relying on experiment lookup
    with mlflow.start_run():
    # Log metrics
        for metric_name, metric_value in evaluation.metrics.items():
            mlflow.log_metric(metric_name, metric_value)

    # Log parameters
        for param_name, param_value in evaluation.params.items():
            mlflow.log_param(param_name, param_value)

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 404 != 200. Response body: ''

In [ ]:
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
mlflow.set_experiment("test-experiment")

In [ ]:
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
mlflow.set_experiment("test-experiment")

with mlflow.start_run():
    mlflow.log_param("param1", 123)
    mlflow.log_metric("metric1", 0.95)

print("✅ Logged successfully!")
